In [3]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


In [5]:
# Load all CSV files
Demand_Data_Normalized = pd.read_csv('Data\ModelData\Demand_YearlyDemandUtilityProfile_Normalized.csv', delimiter=',')
Fuel_Cost_Data_Normalized = pd.read_csv('Data\ModelData\FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Data_Normalized = pd.read_csv('Data\ModelData\VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')
Generation_Asset_Data_Existing = pd.read_csv('Data\ModelData\Generators_AssetData_Existing.csv', delimiter=',')
Demand_Unit_Data = pd.read_csv('Data\ModelData\Demand_UnitSpecificData.csv', delimiter=',')
System_Demand = pd.read_csv('Data\ModelData\System_Demand.csv', delimiter=',')
TransmissionCapacity = pd.read_csv('Data\ModelData\Transmission Capacity.csv', delimiter=',')

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:6: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:6: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
C:\Users\leonr\AppData\Local\Temp\ipykernel_29724\3896660493.py:2: SyntaxWarning: invalid escape sequence '\M'
  Demand_Data_Normalized = pd.read_csv('Data\ModelData\Demand_YearlyDemandUtilityProfile_Normalized.csv', delimiter=',')
C:\Users\leonr\AppData\Local\Temp\ipykernel_29724\3896660493.py:3: S

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\ModelData\\Demand_YearlyDemandUtilityProfile_Normalized.csv'

## Problem 1 - Market clearing as price taker where investment does not influence the market clearing

In [ ]:
#Creating a function for the market clearing

def MarketClearingProblem1(Demand_Data_Normalized,
                           Fuel_Cost_Data_Normalized,
                           Generation_Data_Normalized,
                           Generation_Asset_Data_Existing,
                           Demand_Unit_Data,
                           System_Demand,Season):
    model = gp.Model("MarketClearingProblem1")

    #Defining some iput data

    #Define the number of generators by taking the length of the Generation_Asset_Data_Existing file
    Number_of_Generators = len(Generation_Asset_Data_Existing)
    Number_of_Price_Zones = 2
    Number_of_Demand_Units = len(Demand_Unit_Data)
    Number_of_Hours = 24

    #Data Handling

    #Creating a data frame that contains the demand utility for each demand unit for the season that is modeled
    #Inializing an empty data frame, with 24 hours as columns and number of demand units as rows

    Demand_Utility = pd.DataFrame(index=range(len(Demand_Unit_Data)), columns=[f"Hour{i}" for i in range(Number_of_Hours)])
    
    for a in range(len(Demand_Unit_Data)):
        LoadType = Demand_Unit_Data['LoadType'][a]
        for b in range(len(Demand_Data_Normalized)):
            Season_filtered_Demand_Data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == season]
            Demand_Utility[:,Season_filtered_Demand_Data[LoadType]]
            
                Demand_Unit_Data['Utility'] = Demand_Data_Normalized['U']

    #-----------------------------------------------------------------------------------------------------------------

    #Defining the decision variables

    # Defining a variable for the generators day ahead bid
    Gen_DABid = model.addVars(Number_of_Generators, vtype=GRB.CONTINUOUS, name="Gen_DABid")
     # Defining a variable for the demand day ahead bid
    Dem_DABid = model.addVars(Number_of_Demand_Units, vtype=GRB.CONTINUOUS, name="Dem_DABid")
    # Define voltage angle variables for each zone
    Voltage_Angle = model.addVars(Number_of_Price_Zones, vtype=GRB.CONTINUOUS, name="Voltage_Angle")
    # Define power flow variables (as before, but now dependent on voltage angles)
    Power_Flow = model.addVars(len(Transmission_Capacity), vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Power_Flow")
    
    #-----------------------------------------------------------------------------------------------------------------

    #Defining objective function

    #Objective function that maximizes social welfare
    objective = (gp.quicksum(Generation_Asset_Data_Existing['C_i ($/MWh)'][g] * Gen_DABid[g] for g in range(Number_of_Generators)) 
                - gp.quicksum(Dem['C_i ($/MWh)'][g] * Gen_DABid[g] for g in range(Number_of_Generators)))
    model.setObjective(objective, GRB.MINIMIZE)
    
    # Add slack bus constraint: Fix the voltage angle of node 1 (or any other node) to 0 degrees
    model.addConstr(Voltage_Angle[0] == 0, name="Slack_Bus")




In [13]:
df = pd.DataFrame(index=range(len(Demand_Unit_Data)), columns=[f"Hour{i}" for i in range(Number_of_Hours)])

In [17]:


Season_filtered_Demand_Data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == season]

In [22]:
Season_filtered_Demand_Data['LoadType']

KeyError: 'LoadType'

In [19]:
LoadType = 'D_residential'

In [ ]:
import pandas as pd

# Assume Demand_Unit_Data and Demand_Data_Normalized are already loaded DataFrames
# season_value is provided externally

# Filter Demand_Data_Normalized by the given season
season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == season]

# Create an empty DataFrame to store the results, with 24 columns for each hour
result_data = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

# Loop through each row in Demand_Unit_Data
for _, row in Demand_Unit_Data.iterrows():
    load_type = row['LoadType'].strip()  # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
    percent_load = row['% of system load'] / 100  # Scale factor as a fraction
    
    # Check if the load type exists in the Demand_Data_Normalized columns
    if load_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values = season_data[season_data['Hour'] <= 24][load_type].values * percent_load
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        row_data['Load #'] = row['Load #']
        row_data['Zone'] = row['Zone']
        row_data['LoadType'] = load_type
        
        # Append the new row to the result_data DataFrame
        result_data = pd.concat([result_data, row_data], ignore_index=True)

# Reset index and inspect result
result_data.reset_index(drop=True, inplace=True)
print(result_data)


      Hour_1    Hour_2    Hour_3    Hour_4    Hour_5    Hour_6    Hour_7  \
0   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
1   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
3   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
4   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
6   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
7   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
9   0.734934  0.716599  0.716209  0.731226  0.759359  0.757673  0.758828   
10  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.300000   
12  0.734934

C:\Users\konst\AppData\Local\Temp\ipykernel_17768\4264930273.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_data = pd.concat([result_data, row_data], ignore_index=True)
